In [1]:
# coding: utf-8

# In[ ]:


import requests
import re
import os, csv, sys 
import scipy
import pandas as pd
import gzip
import xml.etree.ElementTree as ET
from os import listdir
from datetime import datetime
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import random
from netCDF4 import Dataset
import itertools
from functions import *

-	Typhoon Hagupit (04 Dec 2014) 13:15 UTC
-	Typhoon Melor (14 Dec 2015)
-	Typhoon Lawin (19 Oct 2016)
-	Typhoon Nock-ten (25 dec 2016)
-	Typhoon Mangkhut (15 sept 2018): first impact map based on UCL forecast was issued on 12 September/ trigger was reached 72h in advance;
-	Typhoon Kammuri (02 dec 2019): first impact map based on UCL was issued on the night of 27 November / trigger was reached 72h in advance;
-	Typhoon Phanfone (24 dec 2019): first impact map based on UCL forecast issued on 21 December / trigger reached in the last 24h;
-	Typhoon Vongfong (14 may 2020): first impact map based on UCL forecast issued on 12 May / trigger reached in the last 24h.



In [2]:
os.getcwd()

'C:\\Users\\ATeklesadik\\OneDrive - Rode Kruis\\Documents\\documents\\Typhoon-Impact-based-forecasting-model\\skill_analysis\\new_code'

In [3]:
df_obs = pd.read_csv(data_obs_folder+'ibtracs.WP.list.v04r00.csv')
#cyclone_names = [x.lower().strip() for x in sorted(list(set(df_obs['NAME'])))]#+['vongfong']
DF=df_obs[(df_obs['LANDFALL'] !=' ' )& (df_obs['NAME'] !='NOT_NAMED') ]
 

cyclone_possible_names = [x.lower().strip() for x in sorted(list(set(DF[DF['SEASON'] >2019 ]['NAME'])))]#
cyclone_names = [x.lower().strip() for x in sorted(list(set(df_obs['NAME'])))]#


NameError: name 'data_obs_folder' is not defined

In [4]:
cyclone_possible_names.append('vamco')

NameError: name 'cyclone_possible_names' is not defined

In [3]:
cyclone_possible_names=['goni']

In [6]:
# In[ ]:
def check_status(file_num, file_num_tot, start):
    percent_complete = (file_num/file_num_tot)*100
    end = time.time()
    sys.stdout.write('\r%.3f %s, %s: %.1f' % (percent_complete, '% Completed', 'time elapsed', end-start))
    sys.stdout.flush()

email = 'ateklesadik@redcross.nl'
pswd = '445273'
data_obs_folder = '../data/obs/'
data_models_folder = '../data/temporarily_downloaded2/'
results_folder = '../data/multicyclone2/'
delete_previous_results_files = 'y'
save_csv_files = 'y'  # one per model


df_obs = pd.read_csv(data_obs_folder+'ibtracs.WP.list.v04r00.csv')
cyclone_names = [x.lower().strip() for x in sorted(list(set(df_obs['NAME'])))]#+['vongfong']

p = {}
p['hagupit'] = ['Hagupit','hagupit', 'reming'] 
p['hagupit'] = ['Hagupit','hagupit', 'reming'] 
p['melor'] = ['Melor','melor', 'reming']
p['haima'] = ['Haima','haima','Lawin', 'reming']
p['kammuri'] = ['Kammuri','kammuri', 'reming']
p['phanfone'] = ['Phanfone','phanfone', 'reming']
p['vongfong'] = ['Vongfong','vongfong', 'reming']
p['nock-ten'] = ['Nockten','nock-ten', 'nockten', 'nock ten', 'nock_ten', 'nina']
p['mangkhut'] = ['mangkhut','mangkhut', 'ompong']

p = {}
#p['jokwe'] = ['Jokwe', 'jokwe']
p['vamco'] = ['vamco','Vamco', 'ulysses','25W','25w','TWENTYFIVE','twentyfive'] 

C:\Users\ATeklesadik\anaconda3\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (19,20,23,24,25,38,39,40,131,132,133,161,162) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
cyclone_possible_names = [p[x] for x in cyclone_names if x in p.keys() ]
cyclone_possible_names=['goni','GONI','Goni']

[['vamco', 'Vamco', 'ulysses', '25W', '25w', 'TWENTYFIVE', 'twentyfive']]

In [8]:
cyclone_names=['goni','GONI','Goni']
cyclone_possible_names=['goni','GONI','Goni']

In [8]:
try:
    import getpass
    input = getpass.getpass
except:
    try:
        input = raw_input
    except:
        pass
    
values = {'email' : email, 'passwd' : pswd, 'action' : 'login'}
login_url = 'https://rda.ucar.edu/cgi-bin/login'

ret = requests.post(login_url, data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print('ret.text')
    exit(1)
    
dspath = 'https://rda.ucar.edu/data/ds330.3/'


In [9]:
year_list = [str(x) if x>9 else '0'+str(x) for x in range(2020, 2021)]
#year_list.remove('2017')
year_list

['2020']

In [10]:
# ## Create filelist to download

# In[ ]:

institute_list = ['RJTD', 'ecmf', 'egrr', 'kwbc']
model_list = [['GSM','GEM','TEM','WFM'],
              ['ifs'],
              ['mogreps'],
              ['CENS','CMC','GEFS','GFS']]
model_spec_list = [[['tcan_nwp','tctr_nwp'],['etctr_nwp'],['etctr_nwp'],['etctr_nwp']],
                   [['all_glo']],
                   [['etctr_glo']],
                   [['etctr_glo','esttr_glo'],['tctr_glo','sttr_glo'],['etctr_glo','esttr_glo'],['tctr_glo','sttr_glo']]]

year_list = [str(x) if x>9 else '0'+str(x) for x in range(2020, 2021)]
#year_list.remove('2017')
month_list = [['11']]
day_list_preliminary = [[[7,12]]]
hour_list = ['00','06','12','18']

day_list = []

for y in day_list_preliminary:
    months = []
    for m in y:
        days = [str(x) if x>9 else '0'+str(x) for x in range(m[0], m[1]+1)]
        months.append(days)
    day_list.append(months)
filelist = []

for i,institute in enumerate(institute_list):
    for mod,model in enumerate(model_list[i]):
        for model_spec in model_spec_list[i][mod]:
            for y,year in enumerate(year_list):
                for m,month in enumerate(month_list[y]):
                    for day in day_list[y][m]:
                        for hour in hour_list:
                            filename = institute.lower()+'/'+year+'/'+year+month+day+'/z_tigge_c_'+institute+'_'+year+month+day+hour+'0000_'+model+'_glob_prod_'+model_spec+'.xml'
                            filelist.append(filename)

In [11]:
filelist

['rjtd/2020/20201107/z_tigge_c_RJTD_20201107000000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201107/z_tigge_c_RJTD_20201107060000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201107/z_tigge_c_RJTD_20201107120000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201107/z_tigge_c_RJTD_20201107180000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201108/z_tigge_c_RJTD_20201108000000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201108/z_tigge_c_RJTD_20201108060000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201108/z_tigge_c_RJTD_20201108120000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201108/z_tigge_c_RJTD_20201108180000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201109/z_tigge_c_RJTD_20201109000000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201109/z_tigge_c_RJTD_20201109060000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201109/z_tigge_c_RJTD_20201109120000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201109/z_tigge_c_RJTD_20201109180000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201110/z_tigge_c_RJTD_2020

In [12]:
import time
start = time.time()
filelist_actual = []

for file_num, file in enumerate(filelist):
    filename = dspath + file
    check_status(file_num, len(filelist), start)
    outfile = data_models_folder + os.path.basename(filename)      
    r = requests.head(filename, cookies = ret.cookies, allow_redirects=False)
    print(r.status_code)
    if r.status_code == 200:
        filelist_actual.append(file)
        req = requests.get(filename, cookies = ret.cookies, allow_redirects=False)
        open(outfile, 'wb').write(req.content)
        CXML_to_csv(outfile, cyclone_names, cyclone_possible_names, results_folder)
        print('extracting')
        #os.remove(outfile)
    else:
        r_gz = requests.head(filename+'.gz', cookies = ret.cookies, allow_redirects=False)
        if r_gz.status_code == 200:
            filelist_actual.append(file+'.gz')
            req_gz = requests.get(filename+'.gz', cookies = ret.cookies, allow_redirects=False)
            open(outfile+'.gz', 'wb').write(req_gz.content)
            CXML_to_csv(outfile+'.gz', cyclone_names, cyclone_possible_names, results_folder)
            #os.remove(outfile+'.gz')

0.000 % Completed, time elapsed: 0.0404
0.278 % Completed, time elapsed: 2.8404
0.556 % Completed, time elapsed: 5.3404
0.833 % Completed, time elapsed: 7.1404
1.111 % Completed, time elapsed: 8.9404
1.389 % Completed, time elapsed: 11.5404
1.667 % Completed, time elapsed: 14.0404
1.944 % Completed, time elapsed: 16.6404
2.222 % Completed, time elapsed: 19.2404
2.500 % Completed, time elapsed: 22.2404
2.778 % Completed, time elapsed: 25.5404
3.056 % Completed, time elapsed: 28.1404
3.333 % Completed, time elapsed: 30.6404
3.611 % Completed, time elapsed: 33.6404
3.889 % Completed, time elapsed: 37.3404
4.167 % Completed, time elapsed: 39.9404
4.444 % Completed, time elapsed: 42.4404
4.722 % Completed, time elapsed: 45.1404
5.000 % Completed, time elapsed: 47.7404
5.278 % Completed, time elapsed: 50.4404
5.556 % Completed, time elapsed: 53.9404
5.833 % Completed, time elapsed: 56.8404
6.111 % Completed, time elapsed: 59.4404
6.389 % Completed, time elapsed: 62.1404
6.667 % Completed, ti

In [151]:

# ## Create filelist to download

# In[ ]:

institute_list = ['RJTD', 'ecmf', 'egrr', 'kwbc']
model_list = [['GSM','GEM','TEM','WFM'],
              ['ifs'],
              ['mogreps'],
              ['CENS','CMC','GEFS','GFS']]
model_spec_list = [[['tcan_nwp','tctr_nwp'],['etctr_nwp'],['etctr_nwp'],['etctr_nwp']],
                   [['all_glo']],
                   [['etctr_glo']],
                   [['etctr_glo','esttr_glo'],['tctr_glo','sttr_glo'],['etctr_glo','esttr_glo'],['tctr_glo','sttr_glo']]]

year_list = [str(x) if x>9 else '0'+str(x) for x in range(2014, 2021)]
year_list.remove('2017')
month_list = [['11','12'],
              ['12'],
              ['10','12'],
              ['09'],
              ['11','12'],
              ['05']]
day_list_preliminary = [[[25,30],[1,7]],
                        [[8,17]],
                        [[12,24],[18,28]],
                        [[8,18]],
                        [[22,30],[1,30]],
                        [[8,20]]]
hour_list = ['00','06','12','18']

day_list = []

for y in day_list_preliminary:
    months = []
    for m in y:
        days = [str(x) if x>9 else '0'+str(x) for x in range(m[0], m[1]+1)]
        months.append(days)
    day_list.append(months)
filelist = []

for i,institute in enumerate(institute_list):
    for mod,model in enumerate(model_list[i]):
        for model_spec in model_spec_list[i][mod]:
            for y,year in enumerate(year_list):
                for m,month in enumerate(month_list[y]):
                    for day in day_list[y][m]:
                        for hour in hour_list:
                            filename = institute.lower()+'/'+year+'/'+year+month+day+'/z_tigge_c_'+institute+'_'+year+month+day+hour+'0000_'+model+'_glob_prod_'+model_spec+'.xml'
                            filelist.append(filename)

In [10]:

len(filelist)

540

In [11]:
filelist

['rjtd/2020/20201107/z_tigge_c_RJTD_20201107000000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201107/z_tigge_c_RJTD_20201107060000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201107/z_tigge_c_RJTD_20201107120000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201107/z_tigge_c_RJTD_20201107180000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201108/z_tigge_c_RJTD_20201108000000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201108/z_tigge_c_RJTD_20201108060000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201108/z_tigge_c_RJTD_20201108120000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201108/z_tigge_c_RJTD_20201108180000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201109/z_tigge_c_RJTD_20201109000000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201109/z_tigge_c_RJTD_20201109060000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201109/z_tigge_c_RJTD_20201109120000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201109/z_tigge_c_RJTD_20201109180000_GSM_glob_prod_tcan_nwp.xml',
 'rjtd/2020/20201110/z_tigge_c_RJTD_2020

In [6]:

path='C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/goni/'

onlyfiles =[]
for path, subdirs, files in os.walk(path):
    for name in files:
        fname=os.path.join(path, name)
        onlyfiles.append(fname)



In [1]:
path='C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/globus/ecmwf/'
import os
onlyfiles =[]
for path, subdirs, files in os.walk(path):
    for name in files:
        fname=os.path.join(path, name)
        onlyfiles.append(fname)

In [9]:
results_folder = '../data/CSVS/'
for file in onlyfiles:
    CXML_to_csv(file, cyclone_names, cyclone_possible_names, results_folder)

In [2]:
onlyfiles

['C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/globus/ecmwf/2006\\20061001\\z_tigge_c_ecmf_20061001000000_ifs_glob_test_all_glo.xml',
 'C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/globus/ecmwf/2006\\20061001\\z_tigge_c_ecmf_20061001120000_ifs_glob_test_all_glo.xml',
 'C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/globus/ecmwf/2006\\20061002\\z_tigge_c_ecmf_20061002000000_ifs_glob_test_all_glo.xml',
 'C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/globus/ecmwf/2006\\20061002\\z_tigge_c_ecmf_20061002120000_ifs_glob_test_all_glo.xml',
 'C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/globus/ecmwf/2006\\20061003\\z_t

In [9]:
results_folder = '../data/CSVS/'
for file in onlyfiles:
    CXML_to_csv(file, cyclone_names, cyclone_possible_names, results_folder)

In [174]:
CXML_to_csv

<function functions.CXML_to_csv(data, cyclone_names, cyclone_possible_names, results_folder)>

In [169]:
cyclone_possible_names

[['hagupit', 'reming'],
 ['haima', 'Lawin', 'reming'],
 ['Kammuri', 'reming'],
 ['mangkhut', 'ompong'],
 ['melor', 'reming'],
 ['nock-ten', 'nockten', 'nock ten', 'nock_ten', 'nina'],
 ['Phanfone', 'reming'],
 ['vongfong', 'reming']]

In [148]:

# ## Create folders

In [33]:
# Create folders

for folder_name in [data_models_folder, results_folder]:
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
# Initialise lists

file_list = []
file_list_short = []
list_failed1 = []
list_total = []
institutes = []
    

# Create list institutes
    
institutes = [x.lower().strip() for x in institute_list]
                 
if delete_previous_results_files == 'y':
    for cyclone_name in cyclone_names:
        try:
            os.remove(results_folder+cyclone_name+'_all.csv')
        except:
            pass

<function functions.CXML_to_csv(data, cyclone_names, cyclone_possible_names, results_folder)>

In [156]:
import time
start = time.time()
filelist_actual = []

for file_num, file in enumerate(filelist_):
    filename = dspath + file
    check_status(file_num, len(filelist), start)
    outfile = data_models_folder + os.path.basename(filename)      
    r = requests.head(filename, cookies = ret.cookies, allow_redirects=False)
    if r.status_code == 200:
        filelist_actual.append(file)
        req = requests.get(filename, cookies = ret.cookies, allow_redirects=False)
        open(outfile, 'wb').write(req.content)
        CXML_to_csv(outfile, cyclone_names, cyclone_possible_names, results_folder)
        #os.remove(outfile)
    else:
        r_gz = requests.head(filename+'.gz', cookies = ret.cookies, allow_redirects=False)
        if r_gz.status_code == 200:
            filelist_actual.append(file+'.gz')
            req_gz = requests.get(filename+'.gz', cookies = ret.cookies, allow_redirects=False)
            open(outfile+'.gz', 'wb').write(req_gz.content)
            CXML_to_csv(outfile+'.gz', cyclone_names, cyclone_possible_names, results_folder)
            #os.remove(outfile+'.gz')

82.182 % Completed, time elapsed: 20161.5

ChunkedEncodingError: ('Connection broken: OSError("(10054, \'WSAECONNRESET\')",)', OSError("(10054, 'WSAECONNRESET')",))

In [84]:
cyclone_names=['goni']
results_folder

'../data/multicyclone/'

In [46]:
cyclone_names

['',
 'abby',
 'abe',
 'abel:beth',
 'aere',
 'agnes',
 'aka',
 'alex',
 'alice',
 'allyn',
 'alma',
 'amber',
 'ampil',
 'amy',
 'amy(-)3',
 'andy',
 'angela',
 'anita',
 'anita:wilda',
 'ann',
 'anna',
 'atsani',
 'axel',
 'babe',
 'babe:babs:carla',
 'babe:carla:charlotte:clara',
 'babs',
 'bailu',
 'banyan',
 'barbara',
 'barijat',
 'bart',
 'bavi',
 'beatrice',
 'bebinca',
 'becky',
 'ben',
 'bernida',
 'bertha',
 'bess',
 'bess:bonnie',
 'beth',
 'betty',
 'beverly',
 'bilie:billie',
 'bilis',
 'bill',
 'billie',
 'bing',
 'bobbie',
 'bolaven',
 'bopha',
 'brenda',
 'brendan',
 'brian',
 'bualoi',
 'caitlin',
 'cam',
 'camilla',
 'carla',
 'carlo',
 'carmen',
 'carol',
 'cary',
 'cass',
 'cathy',
 'cecil',
 'chaba',
 'champi',
 'chan-hom',
 'chanchu',
 'changmi',
 'chanthu',
 'charlott',
 'charlotte',
 'chataan',
 'chebi',
 'chip',
 'choi-wan',
 'chris',
 'chuck',
 'cimaron',
 'clara',
 'colleen',
 'connie',
 'conson',
 'cora',
 'dale',
 'damrey',
 'dan',
 'danas',
 'david',
 'da

In [ ]:
CXML_to_csv(outfile+'.gz', cyclone_names, cyclone_possible_names, results_folder)

In [85]:
path='C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/temporarily_downloadedgoni/'
results_folder='C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/'

onlyfiles = [f for f in listdir(path)]
for files in onlyfiles:
    fname=os.path.join(path,files)
    CXML_to_csv1(fname, cyclone_names, cyclone_possible_names, results_folder)

        
 

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/A

['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/A

['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
['hagupit']
C:/Users/A

[]
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
[]
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
[]
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
[]
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
[]
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
[]
['hagupit']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/hagupit_all.csv
[]
[

...

C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Doc

['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode

C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Doc

['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode Kruis/Documents/documents/Typhoon-Impact-based-forecasting-model/skill_analysis/data/multicyclone/haima_all.csv
['haima']
C:/Users/ATeklesadik/OneDrive - Rode

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['peipah']
['peipah']
['peipah']
['peipah']
['peipah']
['peipah']
['peipah']
['peipah']
['peipah']
['peipah']
['peipah']
['peip

In [76]:
results_folder
def cyclone_in_sublists(cyclone_name, big_list):
    in_big_list = 0
    which_sublist = 0
    for s,sub_list in enumerate(big_list):
        if cyclone_name in sub_list:
            in_big_list = 1
            which_sublist = s
    return in_big_list, which_sublist



In [81]:
cyclone_in_sublists(cyclone_name[0],cyclone_possible_names)[0]

1

In [79]:
cyclone_name=['haima']

In [83]:
cyclone_name and cyclone_in_sublists(cyclone_name[0],cyclone_possible_names)[0] == 1

True

In [84]:
cyclone_possible_names

[['hagupit', 'reming'],
 ['haima', 'Lawin', 'reming'],
 ['Kammuri', 'reming'],
 ['mangkhut', 'ompong'],
 ['melor', 'reming'],
 ['nock-ten', 'nockten', 'nock ten', 'nock_ten', 'nina'],
 ['Phanfone', 'reming'],
 ['vongfong', 'reming']]

In [71]:
def CXML_to_csv1(data, cyclone_names, cyclone_possible_names, results_folder):
    list_failed1 = []
    # Institute name from filename
    institute_name = os.path.basename(data).split('_')[3].lower()
    # Read file
    if data.endswith('.xml'):
        try:
            tree = ET.parse(data)
            root = tree.getroot()
            file_valid = 1
        except:
            file_valid = 0
            pass

    elif data.endswith('.gz'):
        try:
            tree = ET.parse(gzip.open(data))  
            root = tree.getroot()
            file_valid = 1
        except:
            file_valid = 0
            pass

    # Check how many files could not be parsed
    if file_valid == 1:
    
        try:
            model_name=root.find('header/generatingApplication/model/name').text 
        except:
            model_name='NAN'
            pass
        # print(len(list_failed1))
        prod_center=root.find('header/productionCenter').text
        baseTime=root.find('header/baseTime').text

        ## Create one dictonary for each time point, and append it to a list

        for members in root.findall('data'):
    
            Mtype=members.get('type')
     
            for members2 in members.findall('disturbance'):
                try: 
                    cyclone_name = [name.text.lower().strip() for name in members2.findall('cycloneName')]
                    print(cyclone_name)
                except:
                    cyclone_name = [' ']
            
                if cyclone_name and cyclone_in_sublists(cyclone_name[0],cyclone_possible_names)[0] == 1:
            
                    list_data = []
    
                    if Mtype in ['forecast','ensembleForecast']:
                        for members3 in members2.findall('fix'):
                            tem_dic={}
                            tem_dic['Mtype']=[Mtype]
                            tem_dic['institute_name']=[institute_name.lower()]
                            tem_dic['product']=[re.sub('\s+',' ',prod_center).strip().lower()]
                            if model_name != 'NAN':
                                tem_dic['model_name']=[model_name.lower()]
                            else:
                                tem_dic['model_name'] = tem_dic['product']
                            tem_dic['basin'] = [name.text for name in members2.findall('basin')]
                            tem_dic['cycloneName'] = cyclone_name
                            tem_dic['cycloneName2'] = [cyclone_names[cyclone_in_sublists(cyclone_name[0],cyclone_possible_names)[1]]]
                            tem_dic['cycloneNumber'] = [name.text for name in members2.findall('cycloneNumber')]
                            tem_dic['ensemble']=[members.get('member')]#[member]
                            tem_dic['cyc_speed'] = [name.text for name in members3.findall('cycloneData/maximumWind/speed')]
                            #tem_dic['cyc_speed'] = [name.text for name in members3.findall('cycloneData/minimumPressure/pressure')]
                            tem_dic['cyc_cat'] = [name.text for name in members3.findall('cycloneData/development')]
                            time = [name.text for name in members3.findall('validTime')]
                            tem_dic['time'] = ['/'.join(time[0].split('T')[0].split('-'))+', '+time[0].split('T')[1][:-1]]
                            tem_dic['lat'] = [name.text for name in members3.findall('latitude')]
                            tem_dic['lon']= [name.text for name in members3.findall('longitude')]                
                            tem_dic['vhr']=[members3.get('hour')]
                            tem_dic['forecast_time'] = ['/'.join(baseTime.split('T')[0].split('-'))+', '+baseTime.split('T')[1][:-1]]
                            tem_dic1 = dict( [(k,''.join(str(e).lower().strip() for e in v)) for k,v in tem_dic.items()])
                            list_data.append(tem_dic1)
                
                    elif Mtype=='analysis':

                        tem_dic={}
                        tem_dic['Mtype']=['analysis']
                        tem_dic['institute_name']=[institute_name.lower()]
                        tem_dic['product']=[re.sub('\s+',' ',prod_center).strip().lower()]
                        if model_name != 'NAN':
                            tem_dic['model_name']=[model_name.lower()]
                        else:
                            tem_dic['model_name'] = tem_dic['product']
                        tem_dic['basin']= [name.text for name in members2.findall('basin')]
                        tem_dic['cycloneName'] = cyclone_name
                        tem_dic['cycloneName2'] = [cyclone_names[cyclone_in_sublists(cyclone_name[0],cyclone_possible_names)[1]]]
                        tem_dic['cycloneNumber'] = [name.text for name in members2.findall('cycloneNumber')]
                        tem_dic['ensemble']=['NAN']
                        tem_dic['cyc_speed'] = [name.text for name in members2.findall('cycloneData/maximumWind/speed')]
                        #tem_dic['cyc_speed'] = [name.text for name in members2.findall('cycloneData/minimumPressure/pressure')]
                        tem_dic['cyc_cat'] = [name.text for name in members2.findall('cycloneData/development')]
                        time = [name.text for name in members2.findall('fix/validTime')]
                        tem_dic['time'] = ['/'.join(time[0].split('T')[0].split('-'))+', '+time[0].split('T')[1][:-1]]
                        tem_dic['lat'] = [name.text for name in members2.findall('fix/latitude')]
                        tem_dic['lon']= [name.text for name in members2.findall('fix/longitude')]
                        tem_dic['vhr']=[members2.get('hour')]
                        tem_dic['forecast_time'] = ['/'.join(baseTime.split('T')[0].split('-'))+', '+baseTime.split('T')[1][:-1]]
                        tem_dic1 = dict( [(k,''.join(str(e).lower().strip() for e in v)) for k,v in tem_dic.items()])
                        list_data.append(tem_dic1)
                    csv_file = results_folder+cyclone_name[0]+'_all.csv'
                    csv_columns = tem_dic1.keys()
                    print(csv_file)
                    if os.path.exists(csv_file):
                        append_write = 'a' # append if already exists
                    else:
                        append_write = 'w' # make a new file if not

                    try:
                        with open(csv_file, append_write) as csvfile:
                            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
                            if append_write == 'w':
                                writer.writeheader()
                            for row in list_data:
                                writer.writerow(row)
                    except IOError:
                        return "I/O error"

In [59]:
list_failed1 = []
data=fname
# Institute name from filename
institute_name = os.path.basename(fname).split('_')[3].lower()
# Read file
if data.endswith('.xml'):
    try:
        tree = ET.parse(data)
        root = tree.getroot()
        file_valid = 1
    except:
        file_valid = 0
        pass

elif data.endswith('.gz'):
    try:
        tree = ET.parse(gzip.open(data))  
        root = tree.getroot()
        file_valid = 1
    except:
        file_valid = 0
        pass

institute_name

'rjtd'

In [61]:
try:
    model_name=root.find('header/generatingApplication/model/name').text 
except:
    model_name='NAN'
    pass
# print(len(list_failed1))
prod_center=root.find('header/productionCenter').text
baseTime=root.find('header/baseTime').text

## Create one dictonary for each time point, and append it to a list

In [68]:
for members in root.findall('data'):

    Mtype=members.get('type')
    print(Mtype)

    for members2 in members.findall('disturbance'):
        try: 
            cyclone_name = [name.text.lower().strip() for name in members2.findall('cycloneName')]
            print(cyclone_name)
        except:
            cyclone_name = [' ']
            print(cyclone_name)

        if cyclone_name and cyclone_in_sublists(cyclone_name[0],cyclone_possible_names)[0] == 1:

            list_data = []

            if Mtype in ['forecast','ensembleForecast']:
                for members3 in members2.findall('fix'):
                    tem_dic={}
                    tem_dic['Mtype']=[Mtype]
                    tem_dic['institute_name']=[institute_name.lower()]
                    tem_dic['product']=[re.sub('\s+',' ',prod_center).strip().lower()]
                    if model_name != 'NAN':
                        tem_dic['model_name']=[model_name.lower()]
                    else:
                        tem_dic['model_name'] = tem_dic['product']
                    tem_dic['basin'] = [name.text for name in members2.findall('basin')]
                    tem_dic['cycloneName'] = cyclone_name
                    tem_dic['cycloneName2'] = [cyclone_names[cyclone_in_sublists(cyclone_name[0],cyclone_possible_names)[1]]]
                    tem_dic['cycloneNumber'] = [name.text for name in members2.findall('cycloneNumber')]
                    tem_dic['ensemble']=[members.get('member')]#[member]
                    tem_dic['cyc_speed'] = [name.text for name in members3.findall('cycloneData/maximumWind/speed')]
                    #tem_dic['cyc_speed'] = [name.text for name in members3.findall('cycloneData/minimumPressure/pressure')]
                    tem_dic['cyc_cat'] = [name.text for name in members3.findall('cycloneData/development')]
                    time = [name.text for name in members3.findall('validTime')]
                    tem_dic['time'] = ['/'.join(time[0].split('T')[0].split('-'))+', '+time[0].split('T')[1][:-1]]
                    tem_dic['lat'] = [name.text for name in members3.findall('latitude')]
                    tem_dic['lon']= [name.text for name in members3.findall('longitude')]                
                    tem_dic['vhr']=[members3.get('hour')]
                    tem_dic['forecast_time'] = ['/'.join(baseTime.split('T')[0].split('-'))+', '+baseTime.split('T')[1][:-1]]
                    tem_dic1 = dict( [(k,''.join(str(e).lower().strip() for e in v)) for k,v in tem_dic.items()])
                    list_data.append(tem_dic1)

            elif Mtype=='analysis':

                tem_dic={}
                tem_dic['Mtype']=['analysis']
                tem_dic['institute_name']=[institute_name.lower()]
                tem_dic['product']=[re.sub('\s+',' ',prod_center).strip().lower()]
                if model_name != 'NAN':
                    tem_dic['model_name']=[model_name.lower()]
                else:
                    tem_dic['model_name'] = tem_dic['product']
                tem_dic['basin']= [name.text for name in members2.findall('basin')]
                tem_dic['cycloneName'] = cyclone_name
                tem_dic['cycloneName2'] = [cyclone_names[cyclone_in_sublists(cyclone_name[0],cyclone_possible_names)[1]]]
                tem_dic['cycloneNumber'] = [name.text for name in members2.findall('cycloneNumber')]
                tem_dic['ensemble']=['NAN']
                tem_dic['cyc_speed'] = [name.text for name in members2.findall('cycloneData/maximumWind/speed')]
                #tem_dic['cyc_speed'] = [name.text for name in members2.findall('cycloneData/minimumPressure/pressure')]
                tem_dic['cyc_cat'] = [name.text for name in members2.findall('cycloneData/development')]
                time = [name.text for name in members2.findall('fix/validTime')]
                tem_dic['time'] = ['/'.join(time[0].split('T')[0].split('-'))+', '+time[0].split('T')[1][:-1]]
                tem_dic['lat'] = [name.text for name in members2.findall('fix/latitude')]
                tem_dic['lon']= [name.text for name in members2.findall('fix/longitude')]
                tem_dic['vhr']=[members2.get('hour')]
                tem_dic['forecast_time'] = ['/'.join(baseTime.split('T')[0].split('-'))+', '+baseTime.split('T')[1][:-1]]
                tem_dic1 = dict( [(k,''.join(str(e).lower().strip() for e in v)) for k,v in tem_dic.items()])
                list_data.append(tem_dic1)
            csv_file = results_folder+cyclone_name[0]+'_all.csv'
            csv_columns = tem_dic1.keys()

forecast
[]


In [67]:
cyclone_name

[]

In [92]:

                if os.path.exists(csv_file):
                    append_write = 'a' # append if already exists
                else:
                    append_write = 'w' # make a new file if not

                try:
                    with open(csv_file, append_write) as csvfile:
                        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
                        if append_write == 'w':
                            writer.writeheader()
                        for row in list_data:
                            writer.writerow(row)
                except IOError:
                    return "I/O error"

'rjtd'

'2020-11-03T18:00:00Z'

In [111]:
## Create one dictonary for each time point, and append it to a list


In [114]:

                #return "I/O error"